# Airpassenger_ no differencing

In [11]:
!wget -nc https://lazyprogrammer.me/course_files/airline_passengers.csv

--2022-09-13 23:27:22--  https://lazyprogrammer.me/course_files/airline_passengers.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2036 (2.0K) [text/csv]
Saving to: 'airline_passengers.csv'

     0K .                                                     100% 97.7K=0.02s

2022-09-13 23:27:23 (97.7 KB/s) - 'airline_passengers.csv' saved [2036/2036]



In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
df = pd.read_csv('airline_passengers.csv',index_col='Month',parse_dates=True)
df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


## 데이터 전처리

In [25]:
# 로그변환
df['LogPassengers'] = np.log(df['Passengers'])

In [26]:
Ntest = 12
train = df.iloc[:-Ntest]['LogPassengers']
test = df.iloc[-Ntest:]['LogPassengers']

In [27]:
series = df['LogPassengers'].to_numpy()

# 관측 기간
T = 10

X = []
Y = []
for t in range(len(series)-T):
    x = series[t:t+T] # t~t+T 까지 T 과거데이터로
    y = series[t+T] # t+T 데이터 예측
    X.append(x)
    Y.append(y)

In [28]:
X

[array([4.71849887, 4.77068462, 4.88280192, 4.8598124 , 4.79579055,
        4.90527478, 4.99721227, 4.99721227, 4.91265489, 4.77912349]),
 array([4.77068462, 4.88280192, 4.8598124 , 4.79579055, 4.90527478,
        4.99721227, 4.99721227, 4.91265489, 4.77912349, 4.6443909 ]),
 array([4.88280192, 4.8598124 , 4.79579055, 4.90527478, 4.99721227,
        4.99721227, 4.91265489, 4.77912349, 4.6443909 , 4.77068462]),
 array([4.8598124 , 4.79579055, 4.90527478, 4.99721227, 4.99721227,
        4.91265489, 4.77912349, 4.6443909 , 4.77068462, 4.74493213]),
 array([4.79579055, 4.90527478, 4.99721227, 4.99721227, 4.91265489,
        4.77912349, 4.6443909 , 4.77068462, 4.74493213, 4.83628191]),
 array([4.90527478, 4.99721227, 4.99721227, 4.91265489, 4.77912349,
        4.6443909 , 4.77068462, 4.74493213, 4.83628191, 4.94875989]),
 array([4.99721227, 4.99721227, 4.91265489, 4.77912349, 4.6443909 ,
        4.77068462, 4.74493213, 4.83628191, 4.94875989, 4.90527478]),
 array([4.99721227, 4.91265489, 4.

In [31]:
# array로 변환
X = np.array(X).reshape(-1,T)
Y = np.array(Y)
N = len(X)
print('X.shape', X.shape, 'Y.shape', Y.shape)

X.shape (134, 10) Y.shape (134,)


In [32]:
Xtrain, Ytrain = X[:-Ntest], Y[:-Ntest]
Xtest, Ytest = X[-Ntest:], Y[-Ntest:]

## 모델 사용

In [44]:
# 머신러닝 모델
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

lr = LinearRegression()
svr = SVR()
rfr = RandomForestRegressor()

In [48]:
lr.fit(Xtrain,Ytrain)
lr.score(Xtrain,Ytrain)

0.9599443783339897

In [49]:
lr.score(Xtest, Ytest)

0.6910953044565593